In [1]:
# 1. x, t입력
# x : 소리
# 소리를 mfcc 때리면 (소리 길이, 13) 의 자료로 변환
# t : 과일 종류 7개

# 2. 함수 만들면
# Encoder = rnn(13, 13)
# Decoder = Linear(13, 7), decoder의 입력값은, Encoder의 마지막 값 (h)
# 만약 Encoder가 Bidirection = True 면
# h[0:1,:,:]
# h[1:2,:,:] 을 concat으로 이어 붙인 후, reshape 하고 (-1, 26)
# Decoder 입력값도 26개로 늘어남

# 손실함수 : CrossEntropyLoss

In [2]:
from scipy.io import wavfile #소리 불러오는 모듈
import numpy #소리 가공
from python_speech_features import mfcc #mfcc 계산해주는 함수

import torch #AI 모듈
import torch.nn as nn

import os

In [3]:
# 자료 불러오고 전처리 작업

#t 값 가져오기
t_list = os.listdir('data/')
remove_list = []

for t in t_list :
    if t.find('.') >= 0 :
        remove_list.append(t) 

for r in remove_list :
    t_list.remove(r)

print(t_list)
print(t_list.index('kiwi')) #해당 원소가 들어있는 위치 출력하는 함수

['apple', 'banana', 'kiwi', 'lime', 'orange', 'peach', 'pineapple']
2


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

data = []

#x값 가져오기
for t in t_list :
    x_list = os.listdir('data/' + t)
    # print(x_list)
    for x in x_list :
        if x.find('.wav') < 0 : #예외 : .wav 파일이 아니면
            continue
        freq, signal = wavfile.read('data/' + t + '/' + x) # 소리 읽어오는 부분
        x_data = mfcc(signal, freq) #numpy.array 형태의 데이터
        # print(x_data.shape)
        x_tensor = torch.tensor(x_data, dtype = torch.float, device = device).unsqueeze(0) #AI에서 계산하기 위해 바로 텐서로 변환, 3차원으로 만들기 위해 unsqueeze
        t_tensor = torch.tensor(t_list.index(t), dtype = torch.long, device = device)
        temp = (x_tensor, t_tensor) #x값과 t값을 tuple 형태로 묶음, Dataloader를 수동으로 만드는 것과 비슷함
        data.append(temp)
        # print(data)

print(len(data))
print(data[80])

#하나하나씩 데이터를 넣으면 padding작업이 필요없다 (padding은 길이가 다른 자료들을 한번에 계산하기위해 하는 것)

105
(tensor([[[ 1.7211e+01, -2.9088e+01,  1.9201e+00,  1.3996e+01, -1.9930e+01,
           6.9798e-01,  1.5052e+00,  3.8233e+00, -9.2392e+00,  4.3904e+00,
          -7.0654e+00,  1.0714e+01, -9.3964e+00],
         [ 1.9245e+01, -3.8491e+01,  8.1300e+00,  1.4832e+01, -2.5034e+01,
           1.1087e+01, -5.9435e+00,  1.0328e+01, -5.9546e+00, -1.1384e+00,
           2.0243e+00, -3.9229e+00, -6.8235e+00],
         [ 1.9774e+01, -4.8675e+01,  1.1987e+01,  3.8341e+00, -2.4678e+01,
           1.0251e+01,  2.1065e+00,  5.9110e+00, -4.1279e+00,  3.6971e-01,
           8.2902e-01, -5.2163e+00, -1.2364e+00],
         [ 1.9626e+01, -3.1792e+01,  1.8002e+01, -4.1174e+00, -1.2401e+01,
           1.1877e+01, -8.3953e+00,  2.3622e+00, -5.0697e+00,  1.9780e+00,
           1.2417e+00, -1.6478e+00,  1.5543e+00],
         [ 1.8930e+01, -3.7382e+01,  1.9264e+01, -6.1963e+00, -3.1454e+00,
           3.9848e+00, -8.8708e+00,  4.7791e+00, -1.4396e+01,  4.9020e+00,
           4.0223e+00,  2.7591e-01,  4.4067e+

In [6]:
# 함수 만들기
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.encode_rnn = nn.LSTM(13, 13, batch_first = True)
        self.decode_f = nn.Linear(13, 7)
        
    def encode_forward(self, x) :
        y, hc = self.encode_rnn(x)
        return y, hc        
        
    def decode_forward(self, hc) :
        h = hc[0]
        h = h.reshape(-1, 13)
        y = self.decode_f(h)
        return y
        
    def forward(self, x) :
        y, hc = self.encode_forward(x)
        y = self.decode_forward(hc)
        return y
        

F = NN().to(device)
loss_function = nn.CrossEntropyLoss() #분류분제 (네, 아니오)
optimizer = torch.optim.Adam(F.parameters(), lr = 0.001)
epoch = 30

for e in range(epoch) :
    loss_sum = 0
    F.train() #dropout 켜기
    for x, t in data :
        y = F(x)

        loss = loss_function(y, t.unsqueeze(0)) #y는 2차원, t는 1차원
        loss_sum += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_sum /= len(data)

    print(f"epoch {e+1} | loss {loss_sum}")

epoch 1 | loss 1.9409790016355968
epoch 2 | loss 1.6912326279140655
epoch 3 | loss 1.5223653929574148
epoch 4 | loss 1.3541531324386598
epoch 5 | loss 1.1959378883952185
epoch 6 | loss 1.1145559186027163
epoch 7 | loss 0.9340965253966195
epoch 8 | loss 0.8064751187960307
epoch 9 | loss 0.6787914903390975
epoch 10 | loss 0.5687765122879119
epoch 11 | loss 0.4699579969758079
epoch 12 | loss 0.41849132911080406
epoch 13 | loss 0.3445008025992484
epoch 14 | loss 0.3112293142293181
epoch 15 | loss 0.2579952358489945
epoch 16 | loss 0.21960275396704673
epoch 17 | loss 0.18517642571103005
epoch 18 | loss 0.16385415542338574
epoch 19 | loss 0.1463661644075598
epoch 20 | loss 0.13186151651399478
epoch 21 | loss 0.11925309573610623
epoch 22 | loss 0.1077228904480026
epoch 23 | loss 0.09522152308906828
epoch 24 | loss 0.08555724743221488
epoch 25 | loss 0.07678234088456347
epoch 26 | loss 0.0695489440113306
epoch 27 | loss 0.06378445505563701
epoch 28 | loss 0.05873470500643764
epoch 29 | loss 0.

In [ ]:
# # 함수 만들기
# ## Encoder
# class Encoder(nn.Module) :
#     def __init__(self) :
#         super().__init__()
#         self.rnn = nn.LSTM(13, 13, batch_first = True)
#     def forward(self, x) :
#         y, hc = self.rnn(x)
#         return y, hc
        
# ## Decoder 
# class Decoder(nn.Module) :
#     def __init__(self) :
#         super().__init__()
#         self.f = nn.Linear(13, 7)
#     def forward(self, hc) :
#         h = hc[0] # h, c 중에서 h만 골라내기
#         h = h.reshape(-1, 13) # 3차원을 2차원으로 바꾸기
#         y = self.f(h)
#         return y

# encoder = Encoder().to(device)
# decoder = Decoder().to(device)
# loss_function = nn.CrossEntropyLoss() #분류분제 (네, 아니오)
# encoder_optim = torch.optim.Adam(encoder.parameters(), lr = 0.001)
# decoder_optim = torch.optim.Adam(decoder.parameters(), lr = 0.001)
# epoch = 30

# for e in range(epoch) :
#     loss_sum = 0
#     encoder.train()
#     decoder.train() #dropout 켜기
#     for x, t in data :
#         y, hc = encoder(x)
#         y = decoder(hc)

#         loss = loss_function(y, t.unsqueeze(0)) #y는 2차원, t는 1차원
#         loss_sum += loss.item()

#         loss.backward()
#         decoder_optim.step()
#         encoder_optim.step()
#         decoder_optim.zero_grad()
#         encoder_optim.zero_grad()
#     loss_sum /= len(data)

#     print(f"epoch {e+1} | loss {loss_sum}")

In [8]:
## AI 정확도 알아보기

correct = 0

for x, t in data :
    with torch.no_grad() : #자동 미분기능 끄는 함수, 안넣어도 문제는 없지만, 넣으면 살짝 더 빨라집니다.
        y = F(x)
    if y.argmax(dim = -1).item() == t :
        correct += 1
        
acc = correct / len(data)
print(acc)

1.0


In [9]:
#신경망 함수 저장

torch.save(F.to("cpu"), "sound_ai.pt")

# torch.save(encoder.to("cpu"), 'encoder.pt')
# torch.save(decoder.to("cpu"), 'decoder.pt')

In [ ]:
# (문장갯수, 단어갯수, 벡터)
# (소리갯수, 소리길이, 벡터)
# (batchs, features, vector)
# (b, f, v) # batch first는 이렇게 자료를 넣겠다는 뜻이고

# (f, b, v) # batch first가 false일 경우